# **LCOFI Algorithm Analysis on `chess.dat` Dataset**

This document outlines the steps and methodology for applying the **LCOFI (Logic Circuit Optimization Frequent Itemset)** algorithm to the `chess.dat` dataset. The primary objective is to discover **frequent itemsets** and generate **association rules** from the transactional data in a computationally efficient manner using graph-based techniques.

---

## **Steps Involved**

### **Step 1: Load the Dataset**

The `chess.dat` dataset contains transactions where each line represents a set of items (represented as integers). These items correspond to attributes or characteristics of chess games. 

The dataset is loaded and processed into a structured format, where each line is read as a set of items (or a transaction). This transactional data will serve as the input for the LCOFI algorithm.

---

### **Step 2: Represent Transactions as a Bipartite Graph**

The LCOFI algorithm employs a **graph-based representation** of transactions:
- **Nodes**: The graph consists of two types of nodes:
  - Transaction nodes: Representing each transaction uniquely.
  - Item nodes: Representing individual items across all transactions.
- **Edges**: Each edge connects a transaction node to an item node if the item is present in the transaction.

This bipartite representation allows efficient traversal and processing of transactions for frequent itemset mining.

---

### **Step 3: Discover Frequent Itemsets**

Frequent itemsets are identified using the LCOFI algorithm, which includes the following steps:
1. **Initialize Single-Item Frequent Itemsets**:
   - Each item is treated as a single-item candidate, and its support (occurrence frequency) is computed.
   - Items meeting the **minimum support threshold** are retained as frequent single-itemsets.

2. **Iterative Candidate Generation**:
   - Larger candidate itemsets are generated from previously discovered frequent itemsets.
   - For `k`-itemsets, candidates are generated by combining `k-1` frequent itemsets while ensuring all subsets are frequent (Apriori property).

3. **Support Counting**:
   - The support of each candidate itemset is computed by checking its occurrence across transactions.
   - Candidates meeting the **minimum support threshold** are retained as frequent itemsets.

4. **Graph Optimization**:
   - The graph representation is updated dynamically to prune infrequent itemsets and reduce computational overhead.

This iterative process continues until no further frequent itemsets can be generated.

---

### **Step 4: Generate Association Rules**

Once frequent itemsets are identified, **association rules** are generated to uncover relationships between items. These rules are evaluated using the following metrics:
- **Support**: The proportion of transactions containing both the antecedent and consequent of the rule.
- **Confidence**: The probability that a transaction containing the antecedent also contains the consequent.
- **Lift**: A measure of the strength of the rule compared to random chance.

Rules meeting the **minimum confidence threshold** are retained, providing valuable insights into patterns and relationships in the dataset.

---

### **Step 5: Output the Results**

The results include:
1. **Frequent Itemsets**:
   - Sets of items that frequently appear together in the dataset, along with their support values.

2. **Association Rules**:
   - Logical rules derived from the frequent itemsets, showing relationships between items with metrics such as confidence and lift.

---

## **Why LCOFI?**

The LCOFI algorithm is chosen for its efficiency in mining frequent itemsets:
- **Graph-Based Optimization**:
  - By representing transactions as a bipartite graph, the algorithm can dynamically prune infrequent itemsets, reducing computational overhead.
- **Iterative Pruning**:
  - Candidate generation and pruning are performed iteratively, ensuring that only relevant itemsets are evaluated in subsequent steps.
- **Scalability**:
  - The algorithm is well-suited for large datasets like `chess.dat`, where traditional methods like Apriori may face performance bottlenecks due to multiple dataset scans.

---

## **Summary of Steps**

1. **Load Transactions**:
   - Read the dataset and store the transactions in a structured format.

2. **Graph Representation**:
   - Convert transactions into a bipartite graph with transaction and item nodes.

3. **Frequent Itemset Mining**:
   - Use the LCOFI algorithm to discover frequent itemsets based on the **minimum support** threshold.

4. **Association Rule Generation**:
   - Generate rules from frequent itemsets, filtering by the **minimum confidence** threshold.

5. **Result Analysis**:
   - Display frequent itemsets and association rules, providing insights into patterns and relationships in the chess dataset.

---

## **Applications**

Applying the LCOFI algorithm to the `chess.dat` dataset helps uncover patterns such as:
- Common opening strategies used in chess games.
- Relationships between game outcomes (e.g., victory or draw) and specific moves or player ratings.
- Insights into player behavior based on game attributes.

These insights can aid researchers, coaches, and enthusiasts in understanding trends and strategies in chess games.

---

This comprehensive analysis demonstrates the effectiveness of the LCOFI algorithm in mining frequent itemsets and generating association rules, enabling meaningful insights from transactional datasets like `chess.dat`.

In [9]:
import pandas as pd
import itertools
import networkx as nx
from mlxtend.frequent_patterns import association_rules

# Step 1: Load and Preprocess the Dataset
def load_chess_dat(filename):
    """Load and preprocess the .dat file into transactions."""
    transactions = []
    with open(filename, 'r') as file:
        for line in file:
            # Split each line into items and convert to a set
            transaction = set(map(int, line.strip().split()))
            transactions.append(transaction)
    return transactions

# Step 2: LCOFI Algorithm Functions
def generate_candidates(frequent_itemsets, size):
    """Generate candidate itemsets of a specific size."""
    return set(
        frozenset(x) for x in itertools.combinations(set(itertools.chain(*frequent_itemsets)), size)
    )

def count_support(itemsets, transactions, min_support):
    """Count the support of itemsets."""
    support_counts = {item: 0 for item in itemsets}
    for transaction in transactions:
        for item in itemsets:
            if item.issubset(transaction):
                support_counts[item] += 1
    return {
        item: count for item, count in support_counts.items()
        if count / len(transactions) >= min_support
    }

def lcofi(transactions, min_support):
    """LCOFI algorithm for frequent itemsets using graph representation."""
    # Build a bipartite graph
    G = nx.Graph()
    for i, transaction in enumerate(transactions):
        transaction_node = f"Transaction_{i}"  # Transaction nodes are strings
        for item in transaction:
            G.add_edge(transaction_node, item)

    # Generate single-item frequent itemsets
    single_items = {frozenset([node]) for node in G.nodes if isinstance(node, int)}
    frequent_itemsets = count_support(single_items, transactions, min_support)

    all_frequent_itemsets = [frequent_itemsets]

    # Iteratively generate larger itemsets
    k = 2
    while frequent_itemsets:
        candidates = generate_candidates(frequent_itemsets, k)
        frequent_itemsets = count_support(candidates, transactions, min_support)
        if frequent_itemsets:
            all_frequent_itemsets.append(frequent_itemsets)
        k += 1

    return all_frequent_itemsets

# Step 3: Generate Association Rules
def generate_association_rules(frequent_itemsets, transactions, min_confidence):
    """Generate association rules from frequent itemsets."""
    # Flatten frequent itemsets
    flat_itemsets = {}
    for level in frequent_itemsets:
        flat_itemsets.update(level)

    # Prepare DataFrame
    num_transactions = len(transactions)
    data = {
        'itemsets': list(flat_itemsets.keys()),
        'support': [support / num_transactions for support in flat_itemsets.values()]
    }
    frequent_itemsets_df = pd.DataFrame(data)

    # Generate association rules
    rules = association_rules(frequent_itemsets_df, metric="confidence", min_threshold=min_confidence, num_itemsets=num_transactions)
    return rules

# Step 4: Apply LCOFI on Chess Dataset
filename = "Data/chess.dat"  # Replace with the actual path to chess.dat
transactions = load_chess_dat(filename)

# Parameters
min_support = 0.8  # Minimum support threshold
min_confidence = 0.6  # Minimum confidence threshold

# Run LCOFI Algorithm
frequent_itemsets = lcofi(transactions, min_support)

# Generate and Print Association Rules
rules = generate_association_rules(frequent_itemsets, transactions, min_confidence)

# Display Results
print("Frequent Itemsets:")
for k, itemsets in enumerate(frequent_itemsets, start=1):
    print(f"Level {k}:")
    for itemset, support in itemsets.items():
        print(f"  {set(itemset)}: {support / len(transactions):.2f}")

print("\nAssociation Rules:")
print(rules)

Frequent Itemsets:
Level 1:
  {40}: 0.99
  {62}: 0.96
  {52}: 1.00
  {56}: 0.95
  {66}: 0.95
  {25}: 0.89
  {36}: 0.97
  {58}: 1.00
  {48}: 0.94
  {44}: 0.82
  {9}: 0.90
  {60}: 0.99
  {42}: 0.85
  {5}: 0.93
  {34}: 0.95
  {64}: 0.82
  {7}: 0.96
  {3}: 0.89
  {29}: 1.00
Level 2:
  {48, 36}: 0.93
  {40, 5}: 0.92
  {9, 62}: 0.86
  {48, 7}: 0.91
  {64, 29}: 0.82
  {60, 29}: 0.98
  {58, 60}: 0.98
  {56, 5}: 0.88
  {34, 58}: 0.95
  {5, 62}: 0.89
  {25, 7}: 0.86
  {34, 42}: 0.81
  {56, 60}: 0.93
  {48, 66}: 0.90
  {52, 7}: 0.96
  {40, 62}: 0.95
  {29, 62}: 0.95
  {40, 56}: 0.94
  {36, 52}: 0.97
  {42, 7}: 0.81
  {25, 34}: 0.85
  {36, 62}: 0.93
  {48, 3}: 0.84
  {64, 60}: 0.81
  {44, 52}: 0.81
  {42, 52}: 0.85
  {56, 58}: 0.94
  {42, 66}: 0.82
  {3, 36}: 0.86
  {56, 9}: 0.86
  {34, 7}: 0.92
  {66, 62}: 0.90
  {9, 52}: 0.90
  {40, 48}: 0.93
  {66, 52}: 0.94
  {64, 58}: 0.82
  {66, 29}: 0.94
  {56, 3}: 0.85
  {9, 29}: 0.90
  {9, 36}: 0.87
  {66, 36}: 0.91
  {60, 62}: 0.94
  {56, 34}: 0.90
  {3,